In [1]:
# Setup before running notebook:
# In Config.py:
#    IK_SOLVER = "trac_ik"
# In DomainConfig.py:
#    ROBOT_NAME = 'yumi'
#    OPENRAVE_ENV_XML = path to 'keva_housing_stochastic.dae'

In [2]:
import os, sys
sys.path.append('../../')

In [3]:
import Config
from src.Simulators.OpenRaveSimulator import *
from openravepy import *
from tf.transformations import *
from src.OpenraveUtils import *
from prpy.planning import ompl, CBiRRTPlanner, mk

In [4]:
# Load environment
sim = OpenRaveSimulator(Config.OPENRAVE_ENV_XML)
env = sim.env
robot = env.GetRobot(Config.ROBOT_NAME)

In [8]:
# Go to pose
# Pre grasp for picking block from Housing
cur_pose = [-3.06161700e-17,  3.06161700e-17,  1.00000000e+00, -9.37349864e-34, 4.91690896e-01,  3.40889478e-01,  5e-01]
# pose = quat+trans
goal_transform_values = matrixFromPose(cur_pose)


# Yumi works with trac_ik
iksolutions = sim.get_trac_ik_solutions(robot, goal_transform_values, check_collisions=True)

# Set active dof values with any of the iksolutions found
robot.SetActiveDOFValues(iksolutions[0])

In [12]:
# Grasp for picking block from Housing
goal_pose = [-3.06161700e-17,  3.06161700e-17,  1.00000000e+00, -9.37349864e-34, 4.91690896e-01,  3.40889478e-01,  3e-01]
offset = (np.asarray(goal_pose) - np.asarray(cur_pose))[-3:]
distance = np.sqrt(np.sum(offset**2))
direction = offset/distance

In [13]:
# Delta trajectories with CBiRRT 
planner = CBiRRTPlanner(timelimit=10.)
trajectory_object = planner.PlanToEndEffectorOffset(robot, direction, distance,smoothingitrs=10,timelimit=10.)
res = planningutils.RetimeTrajectory(trajectory_object)
trajectory_object = trajectory_object.serialize()

In [14]:
# Execute Trajectory
traj = RaveCreateTrajectory(env, '')
Trajectory.deserialize(traj, trajectory_object)
with robot:
    robot.GetController().SetPath(traj)
robot.WaitForController(0)

True

In [15]:
# Reverse Delta Trajectory
distance = np.sqrt(np.sum(offset**2))
direction = -offset/distance
print(direction)

[-0. -0.  1.]


In [16]:
trajectory_object = planner.PlanToEndEffectorOffset(robot, direction, distance,smoothingitrs=10,timelimit=10.)
res = planningutils.RetimeTrajectory(trajectory_object)
trajectory_object = trajectory_object.serialize()

In [17]:
# Execute Trajectory
traj = RaveCreateTrajectory(env, '')
Trajectory.deserialize(traj, trajectory_object)
with robot:
    robot.GetController().SetPath(traj)
robot.WaitForController(0)

True